In [1]:
# TODO: use `order_no` var

In [36]:
import pandas as pd
from services import get_engine
import graphviz
import numpy as np
import math

pd.options.display.max_columns = None

In [3]:
engine_unf = get_engine(fname='../credentials/.prod_unf')

Connected to bi@prod_unf.


In [40]:
engine_analytics = get_engine(fname='../credentials/.server_analytics')

Connected to Analytics_root@PROD_ANALYTICS.


In [4]:
order_query = '''
-- earned value report.sql
-- get order
SELECT
    order_tbl._LineNo3639 AS orderRowNo
    , order_tbl._Fld35933 AS stage
    , room._Description AS room
    , tech._Description AS tech
    , spec._Description AS spec
    , CAST(spec._IDRRef AS uniqueidentifier) AS specId
    , item._Description AS item
    , CAST(item._IDRRef AS uniqueidentifier) AS itemId
    , order_tbl._Fld3644 AS qty
    , batch._Description AS batch
    , CAST(batch._IDRRef AS uniqueidentifier) AS batchId
FROM 
    _Document164X1 orders
    LEFT JOIN _InfoRg16413 AS numbers ON numbers._Fld16426RRef = orders._IDRRef
    LEFT JOIN _Document164_VT3638X1 AS order_tbl 
        ON order_tbl._Document164_IDRRef = orders._IDRRef
    LEFT JOIN _Reference76 AS item ON item._IDRRef = order_tbl._Fld3640RRef
    LEFT JOIN _Reference79 as tech ON item._Fld1533RRef = tech._IDRRef
    LEFT JOIN _Reference16143 AS room ON room._IDRRef = order_tbl._Fld16181RRef
    LEFT JOIN _Reference89 AS batch ON batch._IDRRef = order_tbl._Fld3643RRef
    LEFT JOIN _Reference16144 AS spec ON order_tbl._Fld16182RRef = spec._IDRRef
WHERE
    numbers._Fld16427 LIKE '13925/13/%'     -- номерЗП
    ORDER BY
        order_tbl._LineNo3639
'''

order = pd.read_sql(order_query, engine_unf)

# Production Orders

In [5]:
prods_query = '''
-- earned value report.sql
-- get prod orders    
SELECT CAST(prod0._IDRRef AS uniqueidentifier) AS prodId,
    prod0._Number AS prodNo,
    item._Description AS item,
    batch._Description AS batch,
    CAST(batch._IDRRef AS uniqueidentifier) AS batchId,
    tech.[_Description] AS tech,
    owner._Description AS owner,
    CAST(owner._IDRRef AS uniqueidentifier) AS ownerId,
    parent_prod.[_Number] AS parentProdNo,
    CAST(parent_prod.[_IDRRef] AS uniqueidentifier) AS parentProdId,
    parent_item.[_Description] AS parentItem,
    parent_tech.[_Description] AS parentTech,
    parent_batch.[_Description] AS parentBatch,
    CAST(parent_batch._IDRRef AS uniqueidentifier) AS parentBatchId
FROM _Document163 AS prod0
    LEFT JOIN _Document164X1 AS ordr ON ordr.[_IDRRef] = prod0.[_Fld3543RRef]
    LEFT JOIN _InfoRg16413 AS ordr_no ON ordr_no._Fld16426RRef = ordr.[_IDRRef]
    LEFT JOIN _Document163_VT3562 AS prod_tbl ON prod0._IDRRef = prod_tbl._Document163_IDRRef
    LEFT JOIN _Reference76 AS item ON item._IDRRef = prod_tbl._Fld3564RRef
    LEFT JOIN _Reference76 AS owner ON owner._IDRRef = prod_tbl._Fld33461RRef
    LEFT JOIN _Reference89 AS batch ON batch._IDRRef = prod_tbl._Fld16259RRef
    LEFT JOIN _Reference79 as tech ON item._Fld1533RRef = tech._IDRRef
    LEFT JOIN _Document163 AS parent_prod ON parent_prod.[_IDRRef] = prod0._Fld3555RRef
    LEFT JOIN _Document163_VT3562 AS parent_prod_tbl ON parent_prod._IDRRef = parent_prod_tbl._Document163_IDRRef
    LEFT JOIN _Reference89 AS parent_batch ON parent_batch._IDRRef = parent_prod_tbl._Fld16259RRef
    LEFT JOIN _Reference76 AS parent_item ON parent_item._IDRRef = parent_prod_tbl._Fld3564RRef
    LEFT JOIN _Reference79 as parent_tech ON parent_item._Fld1533RRef = parent_tech._IDRRef
WHERE ordr_no._Fld16427 LIKE '13925/13/%'
    AND CAST(prod0._Posted AS int) = 1
'''

prods = pd.read_sql(prods_query, engine_unf)

In [6]:
prods =prods.merge(
        order[order.batchId.notnull()][['batchId', 'item', 'orderRowNo', 'spec', 'specId']], 
        how='left',
        on=['item', 'batchId'],
        suffixes=('', '_')
)

In [7]:
prods = prods.merge(
        order[order.batchId.notnull()][['batchId', 'item', 'orderRowNo', 'spec', 'specId']], 
        how='left',
        left_on=['parentItem', 'parentBatchId'],
        right_on=['item', 'batchId'],
        suffixes=('', '_parent')
)
prods.drop(columns='batchId_parent', inplace=True)

In [8]:
prods = prods[[
    'prodId',
    'prodNo',
    'item',
    'batch',
    'batchId',
    'tech',
    'orderRowNo',
    'spec',
    'specId',
    'owner',
    'ownerId',
    'parentProdNo',
    'parentProdId',
    'parentItem',
    'parentTech',
    'parentBatch',
    'parentBatchId',
    'item_parent',
    'orderRowNo_parent',
    'specId_parent',
    'spec_parent'
    ]]

In [9]:
# шаблон не должен подчиняться протяжке
parent_cols = [
    'parentProdNo',
    'parentProdId',
    'parentItem',
    'parentBatch',
    'parentBatchId',
    'item_parent',
    'orderRowNo_parent',
    'spec_parent',
    'parentTech'
]

for col in parent_cols:
    prods[col] = prods.apply(
        lambda r: None if r.tech=='Шаблон' and r.parentTech=='Протяжка'
        else r[col] 
        , axis=1
    )

In [10]:
prods[prods.tech.isin(['Шаблон', 'Протяжка'])]  

,prodId,prodNo,item,batch,batchId,tech,orderRowNo,spec,specId,owner,ownerId,parentProdNo,parentProdId,parentItem,parentTech,parentBatch,parentBatchId,item_parent,orderRowNo_parent,specId_parent,spec_parent
0,0C00D4B9-4D29-C727-11ED-641AC93A3DB6,АСФР-024647,Изготовление шаблона,None,None,Шаблон,NaN,NaN,NaN,иМ-11776,0C00AEB9-4D29-C727-11EC-CBB868DDF025,АСФР-024645,0C00D4B9-4D29-C727-11ED-6404970D9813,Изготовление шаблона,Шаблон,АСФР-004707 от 05.08.2022 № строки: 403 Номенк...,0C00D4B9-4D29-C727-11ED-6404970D9811,Изготовление шаблона,404.0,0C00AEB9-4D29-C727-11EC-CBB868DDF026,иМ-11776
311,0C00D4B9-4D29-C727-11ED-6404970D9813,АСФР-024645,Изготовление шаблона,АСФР-004707 от 05.08.2022 № строки: 403 Номенк...,0C00D4B9-4D29-C727-11ED-6404970D9811,Шаблон,404.0,иМ-11776,0C00AEB9-4D29-C727-11EC-CBB868DDF026,иМ-11776,0C00AEB9-4D29-C727-11EC-CBB868DDF025,None,None,None,None,None,None,None,NaN,0C00AEB9-4D29-C727-11EC-CBB868DDF026,None
573,0C00D4B9-4D29-C727-11ED-6404970D9812,АСФР-024646,иМ-11776,АСФР-004707 от 05.08.2022 № строки: 403 Номенк...,0C00D4B9-4D29-C727-11ED-6404970D9811,Протяжка,403.0,иМ-11776,0C00AEB9-4D29-C727-11EC-CBB868DDF026,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN


In [11]:
# верхние ЗНП МФР выводятся из под тиража
for col in parent_cols:
    prods[col] = prods.apply(
        lambda r: 
        None if r.tech in ['Модельные работы', 'Формовочные работы'] 
                and r.parentTech=='Фиброгипс'
        else r[col] 
        , axis=1
    )

In [12]:
# shall be empty
# if there is no parent then there shall be a `orderRowNo`
fill_row_nos = prods[(prods.parentItem.isnull()) & (prods.orderRowNo.isnull())]    
fill_row_nos.head()

,prodId,prodNo,item,batch,batchId,tech,orderRowNo,spec,specId,owner,ownerId,parentProdNo,parentProdId,parentItem,parentTech,parentBatch,parentBatchId,item_parent,orderRowNo_parent,specId_parent,spec_parent
37,0C00C0B9-4D29-C727-11EC-FE9C2BD3B9A9,АСФР-014907,МР Сложная/мелкая геометрия,None,None,Модельные работы,NaN,NaN,NaN,иДс-6408 СФГ,0C0047B9-4D29-C727-11EB-D5B5F9997B99,None,None,None,None,None,None,None,NaN,0C0047B9-4D29-C727-11EB-D5B5F9997B9A,None
329,0C00C6B9-4D29-C727-11ED-1962FC342CD9,АСФР-018279,"МР Значительная реставрация, гладкотянутые панели",def86e78-9b54-4255-91a3-df74a593368a изделие и...,0C00D1B9-4D29-C727-11ED-55F7D5EE9EEA,Модельные работы,NaN,NaN,NaN,иДс-7164 СФГ,0C00BDB9-4D29-C727-11EC-F327B0C0E15A,None,None,None,None,None,None,None,NaN,0C00BDB9-4D29-C727-11EC-F327B0C0E15B,None
343,0C00C4B9-4D29-C727-11ED-14D02DFFA7DB,АСФР-017737,"ФР Тариф 3, без фанеры (площадь более либо рав...",973208f4-e929-443b-b5e6-ce9cd536a16a изделие и...,0C00C8B9-4D29-C727-11ED-1D56065E0B29,Формовочные работы,NaN,NaN,NaN,иДс-6432 СФГ часть 2,0C00C1B9-4D29-C727-11ED-078787CE028C,None,None,None,None,None,None,None,NaN,0C00C1B9-4D29-C727-11ED-078787CE028D,None
345,0C00C4B9-4D29-C727-11ED-14D03B6CD70A,АСФР-017741,"ФР Тариф 3, без фанеры (площадь более либо рав...",167aa921-6cb0-4df9-bbf6-0a18dd00d363 изделие и...,0C00C8B9-4D29-C727-11ED-1D562875B55D,Формовочные работы,NaN,NaN,NaN,иДс-6433 СФГ часть 2,0C00C1B9-4D29-C727-11ED-07886CED237C,None,None,None,None,None,None,None,NaN,0C00C1B9-4D29-C727-11ED-07886CED237D,None
389,0C00C6B9-4D29-C727-11ED-17FB01FAEEAA,АСФР-017999,"ФР Тариф 3, без фанеры (площадь более либо рав...",5dbefbda-bfa6-4de9-a055-c6399a378110 изделие и...,0C00C6B9-4D29-C727-11ED-18B6A841081F,Формовочные работы,NaN,NaN,NaN,иДс-6835 СФГ,0C009AB9-4D29-C727-11EC-2CF9DEC21351,None,None,None,None,None,None,NaN,NaN,NaN,NaN


In [13]:
# fill missing orderRowNo
for i, row in fill_row_nos.iterrows():
    subset = order[(order.spec==row.owner) & (order.tech==row.tech)]
    if len(subset) != 1:
        print(f'Cannot match order row for prod {row.prodNo} {row["item"]}')
        continue
    prods.at[i, 'orderRowNo'] = subset.orderRowNo.values[0]
    prods.at[i, 'spec'] = subset.spec.values[0]

In [14]:
prods[(prods.parentItem.notnull()) & (prods.orderRowNo.notnull())].tech.unique()    # shall be `Фиброгипс` only

array(['Фиброгипс'], dtype=object)

In [15]:
prods.sample(4)

,prodId,prodNo,item,batch,batchId,tech,orderRowNo,spec,specId,owner,ownerId,parentProdNo,parentProdId,parentItem,parentTech,parentBatch,parentBatchId,item_parent,orderRowNo_parent,specId_parent,spec_parent
134,0C00C0B9-4D29-C727-11ED-02804767F79C,АСФР-015229,"МР Геометрия из тяг, 3D панели",e6e9a7cc-3e4d-4fe3-941c-26b96b20f842 изделие и...,0C00C0B9-4D29-C727-11ED-02804767F79B,Модельные работы,NaN,NaN,NaN,иДс-6401 СФГ,0C0047B9-4D29-C727-11EB-D5B49F746572,АСФР-019511,0C00C9B9-4D29-C727-11ED-247DFF352364,МР Сложная/мелкая геометрия,Модельные работы,АСФР-004707 от 05.08.2022 № строки: 272 Номенк...,0C00C9B9-4D29-C727-11ED-247DFF352363,МР Сложная/мелкая геометрия,272.0,0C0047B9-4D29-C727-11EB-D5B49F746573,иДс-6401 СФГ
405,0C00C4B9-4D29-C727-11ED-1333B326D4BC,АСФР-017419,"ФР Тариф 3, без фанеры (площадь более либо рав...",b54590f0-e3bd-4f40-8c37-e24a71791b03 изделие и...,0C00C4B9-4D29-C727-11ED-1333B326D4BB,Формовочные работы,150.0,иДс-6371 СФГ,0C0047B9-4D29-C727-11EB-D5A019CD8C52,иДс-6371 СФГ,0C0047B9-4D29-C727-11EB-D5A019CD8C51,None,None,None,None,None,None,NaN,NaN,NaN,NaN
160,0C00C6B9-4D29-C727-11ED-17FBB4F1AB28,АСФР-018004,"МР Геометрия из тяг, 3D панели",a231f58b-65b9-46b7-9976-59262fef8908 изделие и...,0C00C6B9-4D29-C727-11ED-17FBB4F1AB27,Модельные работы,NaN,NaN,NaN,иДс-6432 СФГ часть 1,0C004CB9-4D29-C727-11EB-D830985780F6,АСФР-017734,0C00C4B9-4D29-C727-11ED-14D02629FB72,МР Сложная/мелкая геометрия,Модельные работы,АСФР-004707 от 05.08.2022 № строки: 2 Номенкла...,0C00C4B9-4D29-C727-11ED-14D02629FB6F,МР Сложная/мелкая геометрия,2.0,0C004CB9-4D29-C727-11EB-D830985780F7,иДс-6432 СФГ часть 1
225,0C00C4B9-4D29-C727-11ED-131622674AFC,АСФР-017375,"ФР Тариф 3, без фанеры (площадь более либо рав...",63496973-400a-4b86-a242-e33cb96e8104 изделие и...,0C00C4B9-4D29-C727-11ED-131622674AFB,Формовочные работы,NaN,NaN,NaN,иДс-6400 СФГ,0C0047B9-4D29-C727-11EB-D5B4657107AA,АСФР-019510,0C00C9B9-4D29-C727-11ED-247D71889D41,"ФР Тариф 2, без фанеры (площадь менее 0,2мкв)",Формовочные работы,АСФР-004707 от 05.08.2022 № строки: 270 Номенк...,0C00C9B9-4D29-C727-11ED-247D66F0C853,"ФР Тариф 2, без фанеры (площадь менее 0,2мкв)",270.0,0C0047B9-4D29-C727-11EB-D5B4657107AB,иДс-6400 СФГ


# Get budget

In [16]:
specs_list = list(order.specId.unique())
specs_list = [spec for spec in specs_list if pd.notnull(spec)]
specs_list = "', '".join(specs_list)
specs_query = f'''
-- complex_specs.sql
SELECT 
    specs._Description AS spec
    , CAST(specs._IDRRef AS uniqueidentifier) AS specId
    , parent._Description AS owner
    , CAST(parent._IDRRef AS uniqueidentifier) AS ownerId
    , parent_tech._Description AS ownerTech
    , CAST(_LineNo16156 AS int) AS specLineNo
    , CAST(component._IDRRef AS uniqueidentifier) AS componentId
    , component._Description AS component
    , component_tech._Description AS componentTech
    , components_list._Fld36159 AS rate
FROM 
    _Reference16144 AS specs
    LEFT JOIN _Reference76 AS parent ON specs._Fld16369RRef = parent._IDRRef
    LEFT JOIN _Reference79 AS parent_tech ON parent._Fld1533RRef = parent_tech._IDRRef
    LEFT JOIN _Reference16144_VT16155 AS components_list ON components_list._Reference16144_IDRRef  = specs._IDRRef
    LEFT JOIN _Reference76 AS component ON components_list._Fld16157_RRRef = component._IDRRef
    LEFT JOIN _Reference79 AS component_tech ON component._Fld1533RRef = component_tech._IDRRef
WHERE 
    CAST(specs.[_IDRRef] AS uniqueidentifier) IN ('{specs_list}')
ORDER BY 
    CAST(specs.[_Code] AS int)
    , CAST(components_list._LineNo16156 AS int)   -- specLineNo
'''

specs = pd.read_sql(specs_query, engine_unf)

In [17]:
def get_rate(row):
    try:
        if pd.isnull(row.spec):
            return specs[row.itemId==specs.componentId].rate.values[0]
        return specs[(row.itemId==specs.componentId) & (row.specId==specs.specId)].rate.values[0]
    except:
        pass

order['rate'] = order.apply(get_rate, axis=1)

In [18]:
order['budget'] = order.rate * order.qty

In [19]:
assert len(order[order.budget.isnull()])==0, 'Not all order row budget calculated.'

# Execution (Job Tickets)

In [20]:
prod_ids = list(prods.prodId.values)
prod_ids = "', '".join(prod_ids)
jobs_query = f'''
-- @job_tickets_tables.sql
-- pulls & castings shops pay monthly
SELECT
    CAST(DATEADD([YEAR], -2000, ticket._Date_Time) AS date) AS jobDate,
    ticket._Number AS job,
    prod_order.[_Number] AS prodOrderNo,
    CAST(prod_order.[_IDRRef] AS uniqueidentifier) AS prodOrderId,
    item._Description AS item,
    CAST(item._IDRRef AS uniqueidentifier) AS itemId,
    owner.[_Description] AS owner,
    CAST(owner._IDRRef AS uniqueidentifier) AS ownerId,
    tech._Description AS tech,
    CAST(tbl._Fld5276RRef AS uniqueidentifier) AS batchId,
    tbl._Fld5271 AS quantity,
    tbl.[_Fld5273] AS rate,
    CAST(tbl._Fld5275 AS float) AS pay
FROM 
    _Document209 ticket 
    LEFT JOIN _Document209_VT5262 tbl ON ticket._IDRRef = tbl._Document209_IDRRef
    LEFT JOIN _Reference76 item ON tbl._Fld5266RRef = item._IDRRef
    LEFT JOIN _Reference79 tech ON tech._IDRRef = item._Fld1533RRef
    LEFT JOIN _Document163 prod_order ON prod_order._IDRRef = ticket._Fld16366RRef
    LEFT JOIN _Document163_VT3562 AS prod_tables 
        ON prod_order._IDRRef = prod_tables._Document163_IDRRef
        AND tbl._LineNo5263 = prod_tables._LineNo3563
    LEFT JOIN _Reference76 owner ON owner.[_IDRRef] = prod_tables._Fld33461RRef 
WHERE 
    CAST(ticket._Posted AS int) = 1
    AND CAST(item._Fld1527 AS int) > 0     -- item_id
    AND CAST(ticket._Fld16366RRef AS uniqueidentifier) IN ('{prod_ids}')
'''
jobs = pd.read_sql(jobs_query, engine_unf)

In [21]:
assert sum(jobs.batchId.isnull()) == 0, 'Job tickets missing batch.'

In [22]:
assert sum(jobs.prodOrderId.isnull()) == 0, 'Production order is missing in Job ticket.'

In [23]:
def get_order_row(prodId):
    prod = prods[prods.prodId==prodId]
    if len(prod) > 0:
        if pd.notnull(prod.orderRowNo.values[0]):
            return prod.orderRowNo.values[0]
        if pd.notnull(prod.orderRowNo_parent.values[0]):
            return prod.orderRowNo_parent.values[0]
        return get_order_row(prod.parentProdId.values[0])

jobs['orderRowNo'] = jobs.prodOrderId.apply(get_order_row)

In [24]:
assert sum(jobs.orderRowNo.isnull())==0, 'Row no not assigned'

# Calc diff

In [25]:
jobs_grouped = jobs.groupby(by='orderRowNo').agg(sum).pay

C:\Users\kan\AppData\Local\Temp\ipykernel_29532\3006422064.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  jobs_grouped = jobs.groupby(by='orderRowNo').agg(sum).pay


In [26]:
order = order.merge(
    jobs_grouped,
    how='left',
    left_on='orderRowNo',
    right_index=True,
    suffixes=('', '_jobs')
)

In [38]:
assert math.isclose(order.pay.sum(), jobs.pay.sum(), rel_tol=0.001), 'Jobs allocation error.'

In [41]:
order.to_sql(
    name='orderBudgetRpt',
    con=engine_analytics,
    if_exists='replace',
    index=False
)

-1

In [39]:
order.head()

,orderRowNo,stage,room,tech,spec,specId,item,itemId,qty,batch,batchId,rate,budget,pay
0,1.0,4.0,Консоль 2 ярус,Фиброгипс,иДс-6432 СФГ часть 1,0C004CB9-4D29-C727-11EB-D830985780F7,иДс-6432 СФГ часть 1,0C004CB9-4D29-C727-11EB-D830985780F6,8.0,None,None,568.0,4544.0,NaN
1,2.0,4.0,Консоль 2 ярус,Модельные работы,иДс-6432 СФГ часть 1,0C004CB9-4D29-C727-11EB-D830985780F7,МР Сложная/мелкая геометрия,0C00A2B9-4D29-C727-11EC-78FCB338E9FC,1.0,АСФР-004707 от 05.08.2022 № строки: 2 Номенкла...,0C00C4B9-4D29-C727-11ED-14D02629FB6F,73594.0,73594.0,49063.0
2,3.0,4.0,Консоль 2 ярус,Формовочные работы,иДс-6432 СФГ часть 1,0C004CB9-4D29-C727-11EB-D830985780F7,"ФР Тариф 3, без фанеры (площадь более либо рав...",0C00A2B9-4D29-C727-11EC-78FCF1ED9516,1.0,АСФР-004707 от 05.08.2022 № строки: 3 Номенкла...,0C00C4B9-4D29-C727-11ED-14D02629FB74,20443.0,20443.0,NaN
3,4.0,4.0,Консоль 2 ярус,Фиброгипс,иДс-6432 СФГ часть 2,0C00C1B9-4D29-C727-11ED-078787CE028D,иДс-6432 СФГ часть 2,0C00C1B9-4D29-C727-11ED-078787CE028C,8.0,АСФР-004707 от 05.08.2022 № строки: 4 Номенкла...,0C00D4B9-4D29-C727-11ED-619E8EDCB0DB,737.0,5896.0,NaN
4,5.0,4.0,Консоль 2 ярус,Модельные работы,иДс-6432 СФГ часть 2,0C00C1B9-4D29-C727-11ED-078787CE028D,МР Сложная/мелкая геометрия,0C00A2B9-4D29-C727-11EC-78FCB338E9FC,1.0,АСФР-004707 от 05.08.2022 № строки: 5 Номенкла...,0C00C4B9-4D29-C727-11ED-14D02629FB76,10420.0,10420.0,42993.0


# Output

In [111]:
f = graphviz.Digraph(filename="output.gv")
f.attr(rankdir='LR')  
for _, r in prods.sort_values(by='orderRowNo', ascending=False).iterrows():
    match r.tech:
        case 'Фиброгипс':
            f.attr('node', shape='box')
        case 'Модельные работы':
            f.attr('node', shape='cds')
        case 'Формовочные работы':
            f.attr('node', shape='octagon')
        case 'Шаблон':
            f.attr('node', shape='trapezium')
        case 'Протяжка':
            f.attr('node', shape='house')
        case _:
            f.attr('node', shape='')

    if pd.notnull(r.orderRowNo):
        f.attr('node', style='filled', fillcolor='lightblue')
    else:
        f.attr('node', style='', fillcolor='')
    label = (
            (f'c.{r.orderRowNo:.0f}\n' if pd.notnull(r.orderRowNo) else '') +
            f'{r.prodNo}\n' +
            (f'{r.spec}' if pd.notnull(r.spec) else '---') +
            f'\n{r["item"]}'
    )
    f.node(r.prodNo, label)
for _, r in prods[prods.parentProdNo.notnull()].iterrows():
    if (
            r.parentTech == 'Фиброгипс' 
            and r.tech in ['Модельные работы', 'Формовочные работы']
    ): continue
    f.edge(r.parentProdNo, r.prodNo)

f.render()

'output.gv.pdf'